In [11]:
from langchain.chains import RetrievalQA
from langchain.llms import GooglePalm

llm = GooglePalm(google_api_key="", temperature=0.1)

In [12]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='codebasics_faqs.csv', source_column="prompt", encoding='cp1252')

data = loader.load()

In [13]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("What is your refund policy?")

load INSTRUCTOR_Transformer
max_seq_length  512


In [14]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [15]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this 

In [16]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [17]:
chain('Do you provide job assistance and also do you provide job gurantee?')

{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters. The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you prov

In [18]:
chain("do you have javascript course?")

{'query': 'do you have javascript course?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}),
  Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}),
